# Bioclimatic variables

Compute the bioclimatic variables as defined in https://pubs.usgs.gov/ds/691/ds691.pdf.

In [ ]:
import xarray as xr
import matplotlib.pyplot as plt
import numpy as np
from os.path import isfile

In [ ]:
dsspat = xr.open_zarr("https://openstack.cebitec.uni-bielefeld.de:8080/swift/v1/DWDCube/cube_spatial.zarr/")
dstime = xr.open_zarr("https://openstack.cebitec.uni-bielefeld.de:8080/swift/v1/DWDCube/cube_temporal.zarr/")
subset = dstime.isel(Time=range(0, 2*365+1)) #two year subset for faster testing

# Intermediate variables

### Temperature

In [ ]:
# Monthly max:
if isfile("./datasets/tas_mmax.nc"):
    tas_mmax = xr.open_dataarray("./datasets/tas_mmax.nc")
else:
    tas_mmax = dstime["tas"].resample(Time="1MS").max(dim="Time")
    tas_mmax.to_netcdf("./datasets/tas_mmax.nc")
    
    
# Monthly min:
if isfile("./datasets/tas_mmin.nc"):
    tas_mmin = xr.open_dataarray("./datasets/tas_mmin.nc")
else:
    tas_mmin = dstime["tas"].resample(Time="1MS").min(dim="Time")
    tas_mmin.to_netcdf("./datasets/tas_mmin.nc")
    
    
# Monthly mean:
if isfile("./datasets/tas_mmean.nc"):
    tas_mmean = xr.open_dataarray("./datasets/tas_mmean.nc")
else:
    tas_mmean = dstime["tas"].resample(Time="1MS").mean(dim="Time")
    tas_mmean.to_netcdf("./datasets/tas_mmean.nc")

### Precipitation

In [ ]:
# Monthly total:
if isfile("./datasets/prec_mtot.nc"):
    prec_mtot = xr.open_dataarray("./datasets/prec_mtot.nc")
else:
    prec_mtot = dstime["pr"].resample(Time="1MS").sum(dim="Time")
    prec_mtot.to_netcdf("./datasets/prec_mtot.nc")

# [BIO1] Annual Mean Temperature

In [ ]:
ds_bio1 = dstime["tas"].groupby("Time.year").mean("Time")
ds_bio1.to_netcdf("./datasets/bio1.nc")

# [BIO2] Mean Diurnal Range

The mean diurnal range is defined as mean of monthly maximum temperatures and minimum temperatures, i.e.: MDR $= \frac{1}{12} \sum_{i=1}^{12} T_\text{max} - T_\text{min}$

In [ ]:
monthly_range = tas_mmax - tas_mmin

ds_bio2 = monthly_range.groupby("Time.year").mean(dim="Time")
ds_bio2.to_netcdf("./datasets/bio2.nc")

# [BIO3] Isothermality

Defined as: $\frac{\text{BIO2}}{\text{BIO7}} \cdot 100$\
Execute below cell only after computing BIO7...

In [ ]:
ds_bio3 = ds_bio2 / ds_bio7 * 100.
ds_bio3.to_netcdf("./datasets/bio3.nc")

# [BIO4] Temperature Seasonality

Temperature seasonality is defined as the standard deviation (over one year) of monthly temperature averages.

In [ ]:
ds_bio4 = tas_mmean.groupby("Time.year").std(dim="Time")
ds_bio4.to_netcdf("./datasets/bio4.nc")

# [BIO5] Max Temperature of Warmest Month

In [ ]:
ds_bio5 = tas_mmax.groupby("Time.year").max(dim="Time")
ds_bio5.to_netcdf("./datasets/bio5.nc")

# [BIO6] Min Temperature of Coldest Month

In [ ]:
ds_bio6 = tas_mmin.groupby("Time.year").min(dim="Time")
ds_bio6.to_netcdf("./datasets/bio6.nc")

# [BIO7] Temperature Annual Range

Defined as: $\text{BIO5} - \text{BIO6}$

In [ ]:
ds_bio7 = ds_bio5 - ds_bio6
ds_bio7.to_netcdf("./datasets/bio7.nc")

# [BIO8] Mean Temperature of Wettest Quarter

# [BIO9] Mean Temperature of Driest Quarter

# [BIO10] Mean Temperature of Warmest Quarter

# [BIO11] Mean Temperature of Coldest Quarter

# [BIO12] Annual Precipitation

# [BIO13] Precipitation of Wettest Month

# [BIO14] Precipitation of Driest Month

# [BIO15] Precipitation Seasonality

# [BIO16] Precipitation of Wettest Quarter

# [BIO17] Precipitation of Driest Quarter

# [BIO18] Precipitation of Warmest Quarter

# [BIO19] Precipitation of Coldest Quarter